In [41]:
from secret import SECRET
import networkx as nx
import re
import json
import random
from urllib.parse import quote
import requests
import folium as f
import pprint
from demo import DATASET

BASE_URL = "https://maps.googleapis.com/maps/api/directions/json?"
URL_END = "&mode=bicycling&key=" + SECRET
WAYPOINT_MAX = 23 # In case we ever use waypoints
 

In [43]:
import pickle
with open('./data/07' + DATASET + '_no_central_square_final_edges.pkl', 'rb') as file:
    edge_data = pickle.load(file)

with open('./data/07_hardcoded_point_names.pkl', 'rb') as file:
    hardcoded_point_names = pickle.load(file)
    
with open('08' + DATASET + 'previous_urls.pkl', 'rb') as file:
    previous_urls = pickle.load(file)

In [44]:
def decode_polyline(polyline_str):
    '''Pass a Google Maps encoded polyline string; returns list of lat/lon pairs
       WRITTEN BY mgd722
    '''
    index, lat, lng = 0, 0, 0
    coordinates = []
    changes = {'latitude': 0, 'longitude': 0}

    # Coordinates have variable length when encoded, so just keep
    # track of whether we've hit the end of the string. In each
    # while loop iteration, a single coordinate is decoded.
    while index < len(polyline_str):
        # Gather lat/lon changes, store them in a dictionary to apply them later
        for unit in ['latitude', 'longitude']: 
            shift, result = 0, 0

            while True:
                byte = ord(polyline_str[index]) - 63
                index += 1
                result |= (byte & 0x1f) << shift
                shift += 5
                if not byte >= 0x20:
                    break

            if (result & 1):
                changes[unit] = ~(result >> 1)
            else:
                changes[unit] = (result >> 1)

        lat += changes['latitude']
        lng += changes['longitude']

        coordinates.append((lat / 100000.0, lng / 100000.0))

    return coordinates

In [4]:
def generate_url(origin, dest):
    origin_param = "&" + "origin=" +quote((str(origin['lat']) + ',' + str(origin['lon'])))
    dest_param = "&" + "destination=" + quote((str(dest['lat']) + ',' + str(dest['lon'])))

    url = BASE_URL + origin_param + dest_param
    return url



In [5]:
for edge in edge_data:
    origin = {'lat': edge[0][0], 'lon': edge[0][1]}
    dest = {'lat': edge[1][0], 'lon': edge[1][1]}
    
    right = generate_url(origin, dest)
    left = generate_url(dest, origin)
    
    if previous_urls.get(right) is None:
        previous_urls[right] = {} 
    if previous_urls.get(left) is None:
        previous_urls[left] = {}
    

In [6]:
for url, v in previous_urls.items():
    if v.get('response', None) is None:
        print('hit')
        route_data = requests.get(url + URL_END).json()
        previous_urls[url]['response'] = route_data
with open('08' + DATASET + 'previous_urls.pkl', 'wb') as file:
    pickle.dump(previous_urls, file)

In [7]:
unique_nodes = set()

for edge in edge_data:
    origin = (edge[0][0], edge[0][1])
    dest = (edge[1][0], edge[1][1])
    unique_nodes.add(origin)
    unique_nodes.add(dest)


In [32]:
['a'] + ['b']

['a', 'b']

In [45]:
def draw_map():
    m = f.Map(location = [42.3736,-71.109977], zoom_start=14)

    for node in unique_nodes:
        lat = node[0]
        lon = node[1]
        
        size = 25
        color = "green"
        
        node_number = hardcoded_point_names[(lat, lon)]
        
        f.Marker(location=[lat,lon],
          # Set icon to DivIcon to with conditional style formatting to reference the random temp value.
          icon=f.DivIcon(html=f"""<div style="border:1px solid {color}; text-align: center; vertical-align: middle; line-height: {size}px; background-color: white; width: {size}px; height: {size}px; border-radius: {size}px; font-family: sans-serif; color: {color};">{node_number}</div>""")
         ).add_to(m)

    
    for edge in edge_data:
        origin = {'lat': edge[0][0], 'lon': edge[0][1]}
        dest = {'lat': edge[1][0], 'lon': edge[1][1]}
        
        route_to_dest = generate_url(origin, dest)
        rout_to_origin = generate_url(dest, origin)
        
        poly_to_dest = previous_urls[route_to_dest]
        poly_to_origin = previous_urls[route_to_origin]
    
        steps_to_dest = poly_to_dest['response']['routes'][0]['legs'][0]['steps']
        steps_to_origin = poly_to_origin['response']['routes'][0]['legs'][0]['steps']
        
        
        for step in steps_to_dest + steps_to_origin:
            encoded_polyline = step["polyline"]["points"]
            polyline = decode_polyline(encoded_polyline)
#             m.add_child(f.PolyLine(locations=polyline, line_opacity=1, color='#%02X%02X%02X' % (r(),r(),r())))
            m.add_child(f.PolyLine(locations=polyline, opacity=.6, color=color))
 
    return m

In [46]:
m = draw_map()


In [47]:
m

In [31]:
    
for edge in edge_data:
    origin = {'lat': edge[0][0], 'lon': edge[0][1]}
    dest = {'lat': edge[1][0], 'lon': edge[1][1]}

    route_to_dest = generate_url(origin, dest)
    route_to_origin = generate_url(dest, origin)

    poly_to_dest = previous_urls[route_to_dest]
    poly_to_origin = previous_urls[route_to_origin]
    
#     print(poly_to_dest['response'].keys())
    print(poly_to_dest['response']['routes'][0]['legs'][0]['steps'])
    
    break

[{'distance': {'text': '0.2 mi', 'value': 374}, 'duration': {'text': '1 min', 'value': 66}, 'end_location': {'lat': 42.3315965, 'lng': -71.0530672}, 'html_instructions': 'Head <b>northeast</b> on <b>Dorchester St</b> toward <b>Rogers St</b>', 'polyline': {'points': 'qojaG~gupLGQSc@w@uBUk@Us@KWEMiAeDUq@O_@MW_@u@IOWa@KSSYKQ'}, 'start_location': {'lat': 42.3296873, 'lng': -71.0567995}, 'travel_mode': 'BICYCLING'}, {'distance': {'text': '0.5 mi', 'value': 780}, 'duration': {'text': '3 mins', 'value': 176}, 'end_location': {'lat': 42.3379924, 'lng': -71.0569522}, 'html_instructions': 'Turn <b>left</b> onto <b>Old Colony Ave</b>', 'maneuver': 'turn-left', 'polyline': {'points': 'o{jaGtptpLKFaAf@SJaAh@QHC@{@d@}Av@oAp@_Ad@y@b@aD`BaEvBcB|@aFfC]PWH'}, 'start_location': {'lat': 42.3315965, 'lng': -71.0530672}, 'travel_mode': 'BICYCLING'}, {'distance': {'text': '0.2 mi', 'value': 397}, 'duration': {'text': '2 mins', 'value': 93}, 'end_location': {'lat': 42.341539, 'lng': -71.05703760000002}, 'html

In [13]:
m

In [6]:
r = lambda: random.randint(0,255)

m = draw_map()
for step in steps:
    encoded_polyline = step["polyline"]["points"]
    polyline = decode_polyline(encoded_polyline)
#     m.add_child(f.PolyLine(locations=polyline, line_opacity=1, color='#%02X%02X%02X' % (r(),r(),r())))
    m.add_child(f.PolyLine(locations=polyline, opacity=.6, color="magenta"))

m

NameError: name 'draw_map' is not defined